In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import one_hot
from tensorflow.keras.applications import xception
from tensorflow.keras.preprocessing import image
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

**You are provided with a training set and a test set of images of plant seedlings at various stages of grown. Each image has a filename that is its unique id. The dataset comprises 12 plant species. The goal of the competition is to create a classifier capable of determining a plant's species from a photo. The list of species is as follows:**

Black-grass

Charlock

Cleavers

Common Chickweed

Common wheat

Fat Hen

Loose Silky-bent

Maize

Scentless Mayweed

Shepherds Purse

Small-flowered Cranesbill

Sugar beet

In [ ]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
plant_seeds = os.listdir("/kaggle/input/plant-seedlings-classification/train")
plant_seeds

**Using Xception as our pretrained model**

In [ ]:
pretrained = xception.Xception(input_shape=[240,240, 3], include_top=False)

In [ ]:
model = tf.keras.Sequential([
    pretrained,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.BatchNormalization(trainable = True,axis=1),
    
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.BatchNormalization(trainable = True,axis=1),
    
    tf.keras.layers.Dense(12,activation='softmax')
])

**Using the pretrained model to update the weights so setting it as "True"**

In [ ]:
pretrained.trainable = True
model.get_layer('xception').trainable

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.0006),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.4,1],
    rescale=1.0/255.0)

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
path = "/kaggle/input/plant-seedlings-classification/train/"

In [ ]:
result = model.fit(train_datagen.flow_from_directory(path,
                                        target_size=(240, 240), 
                                        color_mode='rgb', 
                                        class_mode='categorical', 
                                        batch_size=64, 
                                        shuffle=True),epochs=10,verbose=1)p

In [ ]:
plt.plot(result.history['accuracy'], label='train', color = 'r')
plt.plot(result.history['val_accuracy'], label='valid', color = 'g')
plt.legend(loc='upper left')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()
plt.plot(result.history['loss'], label='train', color = 'r')
plt.plot(result.history['val_loss'], label='test', color = 'g')
plt.legend(loc='upper right')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()